## Set up your environment

1. Sign up for a [Zep Cloud](https://www.getzep.com/) account.

2. Ensure you install required dependencies into your Python environment before running this notebook. See [Installing Zep SDKs](https://help.getzep.com/sdks) for more information. Optionally create your environment in a `virtualenv`.

```bash
pip install zep-cloud openai rich python-dotenv
```

3. Ensure that you have a `.env` file in your working directory that includes your `ZEP_API_KEY` and `OPENAI_API_KEY`:

```text
ZEP_API_KEY=<key>
OPENAI_API_KEY=<key>
```

> **NOTE:** Zep API keys are specific to a project. You can create multiple keys for a single project. Visit `Project Settings` in the Zep dashboard to manage your API keys.


In [1]:
import os
import json
import uuid

from openai import AsyncOpenAI
import rich

from dotenv import load_dotenv
from zep_cloud.client import AsyncZep
from zep_cloud import Message

load_dotenv()

zep = AsyncZep(api_key=os.environ.get("ZEP_API_KEY"))

oai_client = AsyncOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

## Create User and add a Session

Users in Zep may have one or more chat sessions. These are threads of messages between the user and an agent.

**TIP**: Include the user's full name and email address when creating a user. This improves Zep's ability to associate data, such as emails or documents, with a user.

In [2]:
bot_name = "SupportBot"

user_name = "Emily"
user_id = user_name + str(uuid.uuid4())[:4]
session_id = str(uuid.uuid4())

await zep.user.add(
    user_id=user_id,
    email=f"{user_name}@painters.com",
    first_name=user_name,
    last_name="Painter",
)

await zep.memory.add_session(
    user_id=user_id,
    session_id=session_id,
)

Session(classifications=None, created_at='2024-11-20T03:46:55.999791Z', deleted_at=None, ended_at=None, fact_rating_instruction=None, facts=None, id=38, metadata=None, project_uuid='00000000-0000-0000-0000-000000000000', session_id='217a5813-5d8d-4d77-8e18-151664a26641', updated_at='2024-11-20T03:46:55.999791Z', user_id='Emily34d5', uuid_='425ec4bc-c946-4bdc-b5c6-915a3a7d3c37')

## Datasets

We're going to upload an assortment of data to Zep. These include past dialog with the agent, CRM support cases, and billing data.

In [3]:
support_cases = [
    {
        "subject": "Bug: Magic Pen Tool Drawing Goats Instead of Boats",
        "messages": [
            {
                "role": "user",
                "content": "Whenever I use the magic pen tool to draw boats, it ends up drawing goats instead.",
                "timestamp": "2024-03-16T14:20:00Z",
            },
            {
                "role": "support_agent",
                "content": f"Hi {user_name}, that sounds like a bug! Thanks for reporting it. Could you let me know exactly how you're using the tool when this happens?",
                "timestamp": "2024-03-16T14:22:00Z",
            },
            {
                "role": "user",
                "content": "Sure, I select the magic pen, draw a boat shape, and it just replaces the shape with goats.",
                "timestamp": "2024-03-16T14:25:00Z",
            },
            {
                "role": "support_agent",
                "content": "Got it! We'll escalate this to our engineering team. In the meantime, you can manually select the boat shape from the options rather than drawing it with the pen.",
                "timestamp": "2024-03-16T14:27:00Z",
            },
            {
                "role": "user",
                "content": "Okay, thanks. I hope it gets fixed soon!",
                "timestamp": "2024-03-16T14:30:00Z",
            },
        ],
        "status": "escalated",
    },
]

chat_history = [
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"Hello {user_name}, welcome to PaintWiz support. How can I assist you today?",
        "timestamp": "2024-03-15T10:00:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm absolutely furious! Your AI art generation is completely broken!",
        "timestamp": "2024-03-15T10:02:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I'm sorry to hear that you're experiencing issues, {user_name}. Can you please provide more details about what's going wrong?",
        "timestamp": "2024-03-15T10:03:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's worse, all the people in my drawings have six fingers! It's ridiculous!",
        "timestamp": "2024-03-15T10:05:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I sincerely apologize for the frustration this is causing you, {user_name}. That certainly sounds like a significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me which specific tool or feature you're using when this occurs?",
        "timestamp": "2024-03-15T10:06:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm using the landscape generator and the character creator. Both are completely messed up. How could you let this happen?",
        "timestamp": "2024-03-15T10:08:00Z",
    },
]

transactions = [
    {
        "date": "2024-07-30",
        "amount": 99.99,
        "status": "Success",
        "account_id": user_id,
        "card_last_four": "1234",
    },
    {
        "date": "2024-08-30",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
    {
        "date": "2024-09-15",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
]

account_status = {
    "user_id": user_id,
    "account": {
        "account_id": user_id,
        "account_status": {
            "status": "suspended",
            "reason": "payment failure",
        },
    },
}

def convert_to_zep_messages(chat_history: list[dict[str, str | None]]) -> list[Message]:
    """
    Convert chat history to Zep messages.

    Args:
    chat_history (list): List of dictionaries containing chat messages.

    Returns:
    list: List of Zep Message objects.
    """
    return [
        Message(
            role_type=msg["role"],
            role=msg.get("name", None),
            content=msg["content"],
        )
        for msg in chat_history
    ]

# Zep's high-level API allows us to add a list of messages to a session.
await zep.memory.add(
    session_id=session_id, messages=convert_to_zep_messages(chat_history)
)

# The lower-level data API allows us to add arbitrary data to a user's Knowledge Graph.
for tx in transactions:
    await zep.graph.add(user_id=user_id, data=json.dumps(tx), type="json")

    await zep.graph.add(
        user_id=user_id, data=json.dumps(account_status), type="json"
    )

for case in support_cases:
    await zep.graph.add(user_id=user_id, data=json.dumps(case), type="json")

### Wait a minute or two!

We've batch uploaded a number of datasets that need to be ingested into Zep's graph before they can be queried. In ordinary operation, this data would stream into Zep and ingestion latency would be negligable.

## Retrieve data from Zep

We'll start with getting a list of facts. We'll see the temporal data associated with facts as well as the graph nodes the fact is related to. 

**TIP**: This data is also viewable in the Zep Web application. 

In [4]:
fact_response = await zep.user.get_facts(user_id=user_id)

rich.print(fact_response.facts[:3])


[
    Fact(
        created_at='2024-11-20T03:46:56.722204Z',
        expired_at=None,
        fact='user has the id of Emily34d5',
        invalid_at=None,
        name='HAS_USER',
        rating=None,
        source_node_name='Emily34d5',
        target_node_name='User',
        uuid_='d606171a-b7ac-4cd6-a964-6a679ca129bc',
        valid_at='2024-11-20T03:46:56.722204Z',
        content='user has the id of Emily34d5'
    ),
    Fact(
        created_at='2024-11-20T03:47:02.649211Z',
        expired_at=None,
        fact='SupportBot is assisting Emily in the PaintWiz support.',
        invalid_at=None,
        name='ASSISTS',
        rating=None,
        source_node_name='SupportBot',
        target_node_name='Emily Painter',
        uuid_='8fa5e0d0-6172-46a9-9066-bbd378e7fabf',
        valid_at='2024-11-20T03:46:57.521996Z',
        content='SupportBot is assisting Emily in the PaintWiz support.'
    ),
    Fact(
        created_at='2024-11-20T03:47:02.64926Z',
        expired_at=None,
        fact='Emily is receiving support from PaintWiz support.',
        invalid_at=None,
        name='PROVIDES_SUPPORT',
        rating=None,
        source_node_name='PaintWiz support',
        target_node_name='Emily Painter',
        uuid_='3ab65a1e-8c00-4084-9c49-54cf75873d2f',
        valid_at='2024-11-20T03:46:57.521996Z',
        content='Emily is receiving support from PaintWiz support.'
    )
]

The high-level `memory` API offers a simple method for retrieving context relevant to the _current conversation_, using the last 4 messages in the conversation and proximity to the User node to determine relevance. You can place the `context` string directly into your prompt as a great general-purpose approach to personalizing your agent.

It also returns a list of prior messages for a Session.

In [5]:
m = await zep.memory.get(session_id=session_id)


print(m.context)


FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - A significant glitch in the system is causing disruption to Emily's artistic process. (2024-11-20 03:46:57.521996+00:00 - present)
  - SupportBot is inquiring about the specific feature Emily is using when she encounters the problem. (2024-11-20 03:46:57.521996+00:00 - present)
  - SupportBot is asking Emily which specific tool or feature she is using when the issue occurs. (2024-11-20 03:46:57.521996+00:00 - present)
  - Emily34d5 is experiencing issues with the character creator. (2024-11-20 03:46:57.521996+00:00 - present)
  - Emily34d5 is experiencing issues with the landscape generator. (2024-11-20 03:46:57.521996+00:00 - present)
  - Emily34d5 is using the landscape generator. (2024-11-20 03:46:57.521996+00:00 - present)
  - Emily is experiencing issues with the AI art generation tool. (2024

In [6]:
rich.print(m.messages)

[
    Message(
        content='Hello Emily, welcome to PaintWiz support. How can I assist you today?',
        created_at='2024-11-20T03:46:57.521996Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='236638de-c415-4dec-b0bc-3f2a33957560'
    ),
    Message(
        content="I'm absolutely furious! Your AI art generation is completely broken!",
        created_at='2024-11-20T03:46:57.521996Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='32613d84-95fe-404f-aba6-2f872855a394'
    ),
    Message(
        content="I'm sorry to hear that you're experiencing issues, Emily. Can you please provide more details 
about what's going wrong?",
        created_at='2024-11-20T03:46:57.521996Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='430206ae-3ab4-415a-81b6-9838d88ce986'
    ),
    Message(
        content="Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's 
worse, all the people in my drawings have six fingers! It's ridiculous!",
        created_at='2024-11-20T03:46:57.521996Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='49254394-3f5a-4646-ad96-5e297f75447d'
    ),
    Message(
        content="I sincerely apologize for the frustration this is causing you, Emily. That certainly sounds like a
significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me
which specific tool or feature you're using when this occurs?",
        created_at='2024-11-20T03:46:57.521996Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='b9d430ef-6d89-4881-87d3-3782ae5b3861'
    ),
    Message(
        content="I'm using the landscape generator and the character creator. Both are completely messed up. How 
could you let this happen?",
        created_at='2024-11-20T03:46:57.521996Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='b6778137-343e-4153-8461-48c6eca39ac2'
    )
]

We can also use the `memory` API to search facts for arbritary text. A lower level `graph` search API offers more options, including the ability to search node summaries and various rerankers.

In [7]:
r = await zep.graph.search(user_id=user_id, query="Why are there so many goats?", limit=4, scope="edges")
rich.print([r.fact for r in r.edges])

[
    'When the user draws a boat shape using the magic pen tool, it is replaced with goats.',
    'The app keeps turning mountains into fountains.',
    "All the people in Emily34d5's drawings have six fingers.",
    "A significant glitch in the system is causing disruption to Emily's artistic process."
]

## Creating a simple Chatbot

In the next cells, Emily starts a new chat session with a support agent and complains that she can't log in. Our simple chatbot will, given relevant facts retrieved from Zep's graph, respond accordingly.

Here, the support agent is provided with Emily's billing information and account status, which Zep retrieves as most relevant to Emily's login issue.

In [8]:
new_session_id = str(uuid.uuid4())

emily_message = "Hi, I can't log in!"

await zep.memory.add_session(user_id=user_id, session_id=new_session_id)

# we need to add the Emily's message to the session in order for memory.get to return 
# relevant facts related to the message 
await zep.memory.add(
    session_id=new_session_id,
    messages=[Message(role_type="user", role=user_name, content=emily_message)],
)


SuccessResponse(message='OK')

In [9]:
system_message = """
You are a customer support agent. Carefully review the facts about the user below and respond to the user's question. 
Be helpful and friendly.
"""

memory = await zep.memory.get(session_id=new_session_id)

messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "assistant",
        "content": memory.context,
    },
    {
        "role": "user",
        "content": emily_message,
    },
]

response = await oai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
)

print(response.choices[0].message.content)

Hi Emily! I’m sorry to hear that you’re having trouble logging in. It looks like your account is currently suspended due to a payment failure, which is likely why you can't access it. The issue seems to be related to an expired card.

To resolve this, you’ll need to update your payment information. If you need assistance with that, please let me know, and I can guide you through the process!


In [10]:
print(memory.context)


FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - Emily34d5 is experiencing issues with the character creator. (2024-11-20 03:46:57.521996+00:00 - present)
  - Account with ID 'Emily34d5' has a status of 'suspended'. (2024-11-20 03:46:58.747392+00:00 - present)
  - SupportBot is asking Emily which specific tool or feature she is using when the issue occurs. (2024-11-20 03:46:57.521996+00:00 - present)
  - Emily is experiencing issues with PaintWiz support. (2024-11-20 03:46:57.521996+00:00 - present)
  - Emily34d5 is experiencing issues with the landscape generator. (2024-11-20 03:46:57.521996+00:00 - present)
  - user has the id of Emily34d5 (2024-11-20 03:46:56.722204+00:00 - present)
  - SupportBot is inquiring about the specific feature Emily is using when she encounters the problem. (2024-11-20 03:46:57.521996+00:00 - present)
  - User with 